In [1]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.12.0'

In [2]:
product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)
product_data.head()

,market_ids,clustering_ids,car_ids,firm_ids,region,shares,prices,hpwt,air,mpd,...,supply_instruments2,supply_instruments3,supply_instruments4,supply_instruments5,supply_instruments6,supply_instruments7,supply_instruments8,supply_instruments9,supply_instruments10,supply_instruments11
0,1971,AMGREM71,129,15,US,0.001051,4.935802,0.528997,0,1.888146,...,0.0,1.705933,1.595656,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.888146
1,1971,AMHORN71,130,15,US,0.000670,5.516049,0.494324,0,1.935989,...,0.0,1.680910,1.490295,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.935989
2,1971,AMJAVL71,132,15,US,0.000341,7.108642,0.467613,0,1.716799,...,0.0,1.801067,1.357703,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.716799
3,1971,AMMATA71,134,15,US,0.000522,6.839506,0.426540,0,1.687871,...,0.0,1.818061,1.261347,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.687871
4,1971,AMAMBS71,136,15,US,0.000442,8.928395,0.452489,0,1.504286,...,0.0,1.933210,1.237365,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.504286


In [3]:
X1_formulation = pyblp.Formulation('1 + hpwt + air + mpd + space') ## linear X1 as specified in "random coefficients logit with automobile data"
X2_formulation = pyblp.Formulation('1 + prices + hpwt + air + mpd + space') ## Nonlineaer X2 as specified in "random coefficients logit with automobile data"
product_formulations = (X1_formulation, X2_formulation)
product_formulations

(1 + hpwt + air + mpd + space, 1 + prices + hpwt + air + mpd + space)

In [53]:
agent_data = pd.read_csv(pyblp.data.BLP_AGENTS_LOCATION)
agent_data.head()

,market_ids,weights,nodes0,nodes1,nodes2,nodes3,nodes4,income
0,1971,0.000543,1.192188,0.478777,0.980830,-0.824410,2.473301,109.560369
1,1971,0.000723,1.497074,-2.026204,-1.741316,1.412568,-0.747468,45.457314
2,1971,0.000544,1.438081,0.813280,-1.749974,-1.203509,0.049558,127.146548
3,1971,0.000701,1.768655,-0.177453,0.286602,0.391517,0.683669,22.604045
4,1971,0.000549,0.849970,-0.135337,0.735920,1.036247,-1.143436,170.226032


In [7]:
agent_formulation = pyblp.Formulation('0 + I(1 / income)') ## as specified in "random coefficients logit with automobile data"
agent_formulation

I(1 / income)

In [19]:
blp_problem  = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
blp_problem

Dimensions:
 T    N     F    I     K1    K2    D    MD 
---  ----  ---  ----  ----  ----  ---  ----
20   2217  26   4000   5     6     1    13 

Formulations:
       Column Indices:            0        1      2     3     4      5  
-----------------------------  --------  ------  ----  ---  -----  -----
 X1: Linear Characteristics       1       hpwt   air   mpd  space       
X2: Nonlinear Characteristics     1      prices  hpwt  air   mpd   space
       d: Demographics         1/income                                 

In [34]:
initial_sigma = np.diag([3.612, 0, 4.628, 1.818, 1.050, 2.056]) ## the same as in "random coefficients logit with automobile data"
initial_pi = np.c_[[0, -43.501, 0, 0, 0, 0]] ## the same as in "random coefficients logit with automobile data"

tighter_bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-5}) ## Gradient-based termination tolerance

blp_results = blp_problem.solve(
    initial_sigma,
    initial_pi,
    optimization=tighter_bfgs,
    method='1s'
)
blp_results

Problem Results Summary:
GMM   Objective  Gradient      Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step    Value      Norm    Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  ---------  --------  --------------  --------------  -------  ----------------  -----------------
 1    +3.6E+02   +5.1E-06     +6.5E-01        +5.1E+02        0         +4.1E+07          +1.2E+07     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:20       Yes          17           24          5680         17490   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      1        prices      hpwt        air         mpd        space     |   Pi:     1/income 
------  ----------  --------  ----------  ----------  ----------  ---

In [52]:
initial_sigma2 = np.diag([0.612, 0, 2.318, 0.522, 1.050, 4.246]) ## try some random numbers
initial_pi2 = np.c_[[0, -20.0721, 0, 0, 0, 0]] ## random numbers

tighter_bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-5})
blp_results2 = blp_problem.solve(
    initial_sigma2,
    initial_pi2,
    optimization=tighter_bfgs,
    method='1s'
)
blp_results2

Problem Results Summary:
GMM   Objective  Gradient      Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step    Value      Norm    Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  ---------  --------  --------------  --------------  -------  ----------------  -----------------
 1    +3.6E+02   +1.2E-06     +6.5E-01        +5.0E+02        0         +4.1E+07          +1.9E+07     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:16       Yes          22           27          6374         19613   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      1        prices      hpwt        air         mpd        space     |   Pi:     1/income 
------  ----------  --------  ----------  ----------  ----------  ---